### Fundamentos de Spark Streaming
Imaginemos que queremos procesar un conjunto de datos en streaming que nos llegan en formato JSON. Queremos hacer un recuento del número de eventos por tipo de acción

In [1]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql.functions import *
streamingInputDF = (
    spark
        .readStream
        .schema(jsonSchema)  # Set the schema of the JSON data
        .option('maxFilesPerTrigger', 1) # Treat a sequence of files as a stream by picking one file at a time
        .json(inputPath)
)

In [ ]:
streamingCountsDF = (
    streamingInputDF
        .groupBy(
            streamingInputDF.action,
            window(streamingInputDF.time, '1 hour'))
        .count()
)

In [ ]:
# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

streamingCountsDF es un DataFrame en streaming. Podemos comenzar a aplicar el cálculo definiendo el output de los resultados. En nuestro caso, queremos consultar los recuentos por lo que confugraremos el conjunto completo de recuentos de 1 hora para que esté en una tabla en memoria.

In [ ]:
query = (
    streamingCountDF
            .writeStream
            .format('memory')  # memory = store in-memory-table
            .queryName('counts') # counts = name of the in-memory table
            .outputMode('complete') # complete = all the counts should be in the table
            .start()
)